In [ ]:
# =============================================================
# Pegasus-Large Summarization — MultiNews (Session-only ZIP)
# =============================================================

!pip install transformers datasets rouge-score accelerate --quiet

import torch, gc, os, pandas as pd, sys, zipfile
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# =============================================================
# Config (ONLY paths + dataset adapted)
# =============================================================
MODEL_NAME = "google/pegasus-large"
BATCH_SIZE = 10
CHUNK_SIZE = 200
LIMIT = 15000
SPLIT = "val"

ZIP_PATH = "/content/multi_news.zip"              # 👈 already in session
EXTRACT_DIR = "/content/data_multinews"
SAVE_PATH = f"/content/summaries_pegasus_multinews_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# =============================================================
# Load model + tokenizer safely
# =============================================================
torch.cuda.empty_cache(); gc.collect()
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)

try:
    model = PegasusForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True
    ).to(device)
except RuntimeError:
    print("⚠️ GPU OOM during model load → switching to CPU")
    model = PegasusForConditionalGeneration.from_pretrained(MODEL_NAME).to("cpu")
    device = torch.device("cpu")

model.eval()

# =============================================================
# Load MultiNews from ZIP (session only)
# =============================================================
if not os.path.exists(EXTRACT_DIR):
    print("📦 Extracting MultiNews ZIP...")
    with zipfile.ZipFile(ZIP_PATH, "r") as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("✅ Extraction complete!")
else:
    print("♻️ Using already extracted MultiNews folder.")

# Find src + tgt files
src_file, tgt_file = None, None
for root, _, files in os.walk(EXTRACT_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f and f.endswith(".cleaned"):
            src_file = os.path.join(root, f)
        elif f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

if not src_file or not tgt_file:
    raise ValueError("❌ Could not find MultiNews source/target files")

print(f"✅ Source: {src_file}")
print(f"✅ Target: {tgt_file}")

with open(src_file, "r", encoding="utf-8") as f:
    articles = [l.strip() for l in f.readlines()]

with open(tgt_file, "r", encoding="utf-8") as f:
    references = [l.strip() for l in f.readlines()]

min_len = min(len(articles), len(references))
articles = articles[:min_len][:LIMIT]
references = references[:min_len][:LIMIT]

print(f"📚 Loaded {len(articles)} MultiNews samples ({SPLIT})")

# =============================================================
# Summarization helper (UNCHANGED)
# =============================================================
def summarize_batch(texts):
    inputs = tokenizer(
        texts, max_length=1024, truncation=True, padding=True, return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=130,
            min_length=30,
            num_beams=4,
            early_stopping=True
        )

    outputs = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    del inputs, summary_ids
    torch.cuda.empty_cache(); gc.collect()
    return outputs

# =============================================================
# Save progress locally (UNCHANGED)
# =============================================================
def save_progress(articles, refs, summaries, save_path):
    df = pd.DataFrame({
        "document": articles,
        "reference": refs,
        "summary": summaries
    })
    header = not os.path.exists(save_path)
    df.to_csv(save_path, mode="a", header=header, index=False)

# =============================================================
# Resume support (UNCHANGED)
# =============================================================
start_idx = 0
if os.path.exists(SAVE_PATH):
    df_prev = pd.read_csv(SAVE_PATH)
    start_idx = len(df_prev)
    print(f"🔁 Resuming from index {start_idx}")

# =============================================================
# Main summarization loop (UNCHANGED)
# =============================================================
total_chunks = (len(articles) - start_idx + CHUNK_SIZE - 1) // CHUNK_SIZE
chunk_no = 1

for i in range(start_idx, len(articles), CHUNK_SIZE):
    end = min(i + CHUNK_SIZE, len(articles))
    print(f"\n🚀 Chunk {chunk_no}/{total_chunks} → Samples {i}–{end}")

    batch_articles = articles[i:end]
    batch_refs = references[i:end]
    all_summaries = []

    total_batches = (len(batch_articles) + BATCH_SIZE - 1) // BATCH_SIZE

    for j in range(0, len(batch_articles), BATCH_SIZE):
        sub_batch = batch_articles[j:j + BATCH_SIZE]
        batch_no = (j // BATCH_SIZE) + 1

        try:
            batch_summaries = summarize_batch(sub_batch)
        except torch.cuda.OutOfMemoryError:
            torch.cuda.empty_cache(); gc.collect()
            batch_summaries = summarize_batch(sub_batch)

        all_summaries.extend(batch_summaries)
        sys.stdout.write(f"\r[Batch {batch_no}/{total_batches}]")
        sys.stdout.flush()

    print()
    save_progress(batch_articles, batch_refs, all_summaries, SAVE_PATH)
    print(f"💾 Saved chunk {chunk_no}/{total_chunks} ({end}/{len(articles)} done)")

    chunk_no += 1
    torch.cuda.empty_cache(); gc.collect()

print("\n🎉 All MultiNews summaries complete!")
print("📄 Saved to:", SAVE_PATH)


✅ Using device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📦 Extracting MultiNews ZIP...
✅ Extraction complete!
✅ Source: /content/data_multinews/multi_news/val.src.cleaned
✅ Target: /content/data_multinews/multi_news/val.tgt
📚 Loaded 5622 MultiNews samples (val)

🚀 Chunk 1/29 → Samples 0–200
[Batch 5/20]

KeyboardInterrupt: 

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (SINGLE CELL)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "val"
LIMIT = 200
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i, sample in enumerate(articles):
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")
    torch.cuda.empty_cache(); gc.collect()

# ---------------- SAVE ----------------
pd.DataFrame(results).to_csv(SAVE_PATH, index=False)
print("Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ 1/200 done
✓ 2/200 done
✓ 3/200 done
✓ 4/200 done
✓ 5/200 done
✓ 6/200 done
✓ 7/200 done
✓ 8/200 done
✓ 9/200 done
✓ 10/200 done
✓ 11/200 done
✓ 12/200 done
✓ 13/200 done
✓ 14/200 done


KeyboardInterrupt: 

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "val"
LIMIT = 200
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔁 Resuming from index 200
✅ Summarization complete. Saved to: /content/pegasus_multidoc_val.csv


In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "val"
LIMIT = 400
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔁 Resuming from index 200
✓ 201/400 done
✓ 202/400 done
✓ 203/400 done
✓ 204/400 done
✓ 205/400 done
✓ 206/400 done
✓ 207/400 done
✓ 208/400 done
✓ 209/400 done
✓ 210/400 done
✓ 211/400 done
✓ 212/400 done
✓ 213/400 done
✓ 214/400 done
✓ 215/400 done
✓ 216/400 done
✓ 217/400 done
✓ 218/400 done
✓ 219/400 done
✓ 220/400 done
✓ 221/400 done
✓ 222/400 done
✓ 223/400 done
✓ 224/400 done
✓ 225/400 done
✓ 226/400 done
✓ 227/400 done
✓ 228/400 done
✓ 229/400 done
✓ 230/400 done
✓ 231/400 done
✓ 232/400 done
✓ 233/400 done
✓ 234/400 done
✓ 235/400 done
✓ 236/400 done
✓ 237/400 done
✓ 238/400 done
✓ 239/400 done
✓ 240/400 done
✓ 241/400 done
✓ 242/400 done
✓ 243/400 done
✓ 244/400 done
✓ 245/400 done
✓ 246/400 done
✓ 247/400 done
✓ 248/400 done
✓ 249/400 done
✓ 250/400 done
✓ 251/400 done
✓ 252/400 done
✓ 253/400 done
✓ 254/400 done
✓ 255/400 done
✓ 256/400 done
✓ 257/400 done
✓ 258/400 done
✓ 259/400 done
✓ 260/400 done
✓ 261/400 done
✓ 262/400 done
✓ 263/400 done
✓ 264/400 done
✓ 265/400 done

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "val"
LIMIT = 800
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔁 Resuming from index 400
✓ 401/800 done
✓ 402/800 done
✓ 403/800 done
✓ 404/800 done
✓ 405/800 done
✓ 406/800 done
✓ 407/800 done
✓ 408/800 done
✓ 409/800 done
✓ 410/800 done
✓ 411/800 done
✓ 412/800 done
✓ 413/800 done
✓ 414/800 done
✓ 415/800 done
✓ 416/800 done
✓ 417/800 done
✓ 418/800 done
✓ 419/800 done
✓ 420/800 done
✓ 421/800 done
✓ 422/800 done
✓ 423/800 done
✓ 424/800 done
✓ 425/800 done
✓ 426/800 done
✓ 427/800 done
✓ 428/800 done
✓ 429/800 done
✓ 430/800 done
✓ 431/800 done
✓ 432/800 done
✓ 433/800 done
✓ 434/800 done
✓ 435/800 done
✓ 436/800 done
✓ 437/800 done
✓ 438/800 done
✓ 439/800 done
✓ 440/800 done
✓ 441/800 done
✓ 442/800 done
✓ 443/800 done
✓ 444/800 done
✓ 445/800 done
✓ 446/800 done
✓ 447/800 done
✓ 448/800 done
✓ 449/800 done
✓ 450/800 done
✓ 451/800 done
✓ 452/800 done
✓ 453/800 done
✓ 454/800 done
✓ 455/800 done
✓ 456/800 done
✓ 457/800 done
✓ 458/800 done
✓ 459/800 done
✓ 460/800 done
✓ 461/800 done
✓ 462/800 done
✓ 463/800 done
✓ 464/800 done
✓ 465/800 done

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "val"
LIMIT = 900
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔁 Resuming from index 800
✓ 801/900 done
✓ 802/900 done
✓ 803/900 done
✓ 804/900 done
✓ 805/900 done
✓ 806/900 done
✓ 807/900 done
✓ 808/900 done
✓ 809/900 done
✓ 810/900 done
✓ 811/900 done
✓ 812/900 done
✓ 813/900 done
✓ 814/900 done
✓ 815/900 done
✓ 816/900 done
✓ 817/900 done
✓ 818/900 done
✓ 819/900 done
✓ 820/900 done
✓ 821/900 done
✓ 822/900 done
✓ 823/900 done
✓ 824/900 done
✓ 825/900 done
✓ 826/900 done
✓ 827/900 done
✓ 828/900 done
✓ 829/900 done
✓ 830/900 done
✓ 831/900 done
✓ 832/900 done
✓ 833/900 done
✓ 834/900 done
✓ 835/900 done
✓ 836/900 done
✓ 837/900 done
✓ 838/900 done
✓ 839/900 done
✓ 840/900 done
✓ 841/900 done
✓ 842/900 done
✓ 843/900 done
✓ 844/900 done
✓ 845/900 done
✓ 846/900 done
✓ 847/900 done
✓ 848/900 done
✓ 849/900 done
✓ 850/900 done
✓ 851/900 done
✓ 852/900 done
✓ 853/900 done
✓ 854/900 done
✓ 855/900 done
✓ 856/900 done
✓ 857/900 done
✓ 858/900 done
✓ 859/900 done
✓ 860/900 done
✓ 861/900 done
✓ 862/900 done
✓ 863/900 done
✓ 864/900 done
✓ 865/900 done

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "val"
LIMIT = 1000
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔁 Resuming from index 900
✓ 901/1000 done
✓ 902/1000 done
✓ 903/1000 done
✓ 904/1000 done
✓ 905/1000 done
✓ 906/1000 done
✓ 907/1000 done
✓ 908/1000 done
✓ 909/1000 done
✓ 910/1000 done
✓ 911/1000 done
✓ 912/1000 done
✓ 913/1000 done
✓ 914/1000 done
✓ 915/1000 done
✓ 916/1000 done
✓ 917/1000 done
✓ 918/1000 done
✓ 919/1000 done
✓ 920/1000 done
✓ 921/1000 done
✓ 922/1000 done
✓ 923/1000 done
✓ 924/1000 done
✓ 925/1000 done
✓ 926/1000 done
✓ 927/1000 done
✓ 928/1000 done
✓ 929/1000 done
✓ 930/1000 done
✓ 931/1000 done
✓ 932/1000 done
✓ 933/1000 done
✓ 934/1000 done
✓ 935/1000 done
✓ 936/1000 done
✓ 937/1000 done
✓ 938/1000 done
✓ 939/1000 done
✓ 940/1000 done
✓ 941/1000 done
✓ 942/1000 done
✓ 943/1000 done
✓ 944/1000 done
✓ 945/1000 done
✓ 946/1000 done
✓ 947/1000 done
✓ 948/1000 done
✓ 949/1000 done
✓ 950/1000 done
✓ 951/1000 done
✓ 952/1000 done
✓ 953/1000 done
✓ 954/1000 done
✓ 955/1000 done
✓ 956/1000 done
✓ 957/1000 done
✓ 958/1000 done
✓ 959/1000 done
✓ 960/1000 done
✓ 961/1000 don

test

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "test"
LIMIT = 200
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ 1/200 done
✓ 2/200 done
✓ 3/200 done
✓ 4/200 done
✓ 5/200 done
✓ 6/200 done
✓ 7/200 done
✓ 8/200 done
✓ 9/200 done
✓ 10/200 done
✓ 11/200 done
✓ 12/200 done
✓ 13/200 done
✓ 14/200 done
✓ 15/200 done
✓ 16/200 done
✓ 17/200 done
✓ 18/200 done
✓ 19/200 done
✓ 20/200 done
✓ 21/200 done
✓ 22/200 done
✓ 23/200 done
✓ 24/200 done
✓ 25/200 done
✓ 26/200 done
✓ 27/200 done
✓ 28/200 done
✓ 29/200 done
✓ 30/200 done
✓ 31/200 done
✓ 32/200 done
✓ 33/200 done
✓ 34/200 done
✓ 35/200 done
✓ 36/200 done
✓ 37/200 done
✓ 38/200 done
✓ 39/200 done
✓ 40/200 done
✓ 41/200 done
✓ 42/200 done
✓ 43/200 done
✓ 44/200 done
✓ 45/200 done
✓ 46/200 done
✓ 47/200 done
✓ 48/200 done
✓ 49/200 done
✓ 50/200 done
✓ 51/200 done
✓ 52/200 done
✓ 53/200 done
✓ 54/200 done
✓ 55/200 done
✓ 56/200 done
✓ 57/200 done
✓ 58/200 done
✓ 59/200 done
✓ 60/200 done
✓ 61/200 done
✓ 62/200 done
✓ 63/200 done
✓ 64/200 done
✓ 65/200 done
✓ 66/200 done
✓ 67/200 done
✓ 68/200 done
✓ 69/200 done
✓ 70/200 done
✓ 71/200 done
✓ 72/200 done
✓

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "test"
LIMIT = 400
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔁 Resuming from index 200
✓ 201/400 done
✓ 202/400 done
✓ 203/400 done
✓ 204/400 done
✓ 205/400 done
✓ 206/400 done
✓ 207/400 done
✓ 208/400 done
✓ 209/400 done
✓ 210/400 done
✓ 211/400 done
✓ 212/400 done
✓ 213/400 done
✓ 214/400 done
✓ 215/400 done
✓ 216/400 done
✓ 217/400 done
✓ 218/400 done
✓ 219/400 done
✓ 220/400 done
✓ 221/400 done
✓ 222/400 done
✓ 223/400 done
✓ 224/400 done
✓ 225/400 done
✓ 226/400 done
✓ 227/400 done
✓ 228/400 done
✓ 229/400 done
✓ 230/400 done
✓ 231/400 done
✓ 232/400 done
✓ 233/400 done
✓ 234/400 done
✓ 235/400 done
✓ 236/400 done
✓ 237/400 done
✓ 238/400 done
✓ 239/400 done
✓ 240/400 done
✓ 241/400 done
✓ 242/400 done
✓ 243/400 done
✓ 244/400 done
✓ 245/400 done
✓ 246/400 done
✓ 247/400 done
✓ 248/400 done
✓ 249/400 done
✓ 250/400 done
✓ 251/400 done
✓ 252/400 done
✓ 253/400 done
✓ 254/400 done
✓ 255/400 done
✓ 256/400 done
✓ 257/400 done
✓ 258/400 done
✓ 259/400 done
✓ 260/400 done
✓ 261/400 done
✓ 262/400 done
✓ 263/400 done
✓ 264/400 done
✓ 265/400 done

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "test"
LIMIT = 600
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔁 Resuming from index 400
✓ 401/600 done
✓ 402/600 done
✓ 403/600 done
✓ 404/600 done
✓ 405/600 done
✓ 406/600 done
✓ 407/600 done
✓ 408/600 done
✓ 409/600 done
✓ 410/600 done
✓ 411/600 done
✓ 412/600 done
✓ 413/600 done
✓ 414/600 done
✓ 415/600 done
✓ 416/600 done
✓ 417/600 done
✓ 418/600 done
✓ 419/600 done
✓ 420/600 done
✓ 421/600 done
✓ 422/600 done
✓ 423/600 done
✓ 424/600 done
✓ 425/600 done
✓ 426/600 done
✓ 427/600 done
✓ 428/600 done
✓ 429/600 done
✓ 430/600 done
✓ 431/600 done
✓ 432/600 done
✓ 433/600 done
✓ 434/600 done
✓ 435/600 done
✓ 436/600 done
✓ 437/600 done
✓ 438/600 done
✓ 439/600 done
✓ 440/600 done
✓ 441/600 done
✓ 442/600 done
✓ 443/600 done
✓ 444/600 done
✓ 445/600 done
✓ 446/600 done
✓ 447/600 done
✓ 448/600 done
✓ 449/600 done
✓ 450/600 done
✓ 451/600 done
✓ 452/600 done
✓ 453/600 done
✓ 454/600 done
✓ 455/600 done
✓ 456/600 done
✓ 457/600 done
✓ 458/600 done
✓ 459/600 done
✓ 460/600 done
✓ 461/600 done
✓ 462/600 done
✓ 463/600 done
✓ 464/600 done
✓ 465/600 done

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "test"
LIMIT = 800
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔁 Resuming from index 600
✓ 601/800 done
✓ 602/800 done
✓ 603/800 done
✓ 604/800 done
✓ 605/800 done
✓ 606/800 done
✓ 607/800 done
✓ 608/800 done
✓ 609/800 done
✓ 610/800 done
✓ 611/800 done
✓ 612/800 done
✓ 613/800 done
✓ 614/800 done
✓ 615/800 done
✓ 616/800 done
✓ 617/800 done
✓ 618/800 done
✓ 619/800 done
✓ 620/800 done
✓ 621/800 done
✓ 622/800 done
✓ 623/800 done
✓ 624/800 done
✓ 625/800 done
✓ 626/800 done
✓ 627/800 done
✓ 628/800 done
✓ 629/800 done
✓ 630/800 done
✓ 631/800 done
✓ 632/800 done
✓ 633/800 done
✓ 634/800 done
✓ 635/800 done
✓ 636/800 done
✓ 637/800 done
✓ 638/800 done
✓ 639/800 done
✓ 640/800 done
✓ 641/800 done
✓ 642/800 done
✓ 643/800 done
✓ 644/800 done
✓ 645/800 done
✓ 646/800 done
✓ 647/800 done
✓ 648/800 done
✓ 649/800 done
✓ 650/800 done
✓ 651/800 done
✓ 652/800 done
✓ 653/800 done
✓ 654/800 done
✓ 655/800 done
✓ 656/800 done
✓ 657/800 done
✓ 658/800 done
✓ 659/800 done
✓ 660/800 done
✓ 661/800 done
✓ 662/800 done
✓ 663/800 done
✓ 664/800 done
✓ 665/800 done

In [ ]:
# =============================================================
# TRUE MULTI-DOCUMENT SUMMARIZATION — PEGASUS (RESUMABLE)
# Multi-News | Hierarchical: Doc-level → Fusion
# =============================================================

!pip install transformers rouge-score accelerate --quiet

import torch, gc, os, pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# ---------------- CONFIG ----------------
MODEL_NAME = "google/pegasus-large"
DATA_DIR = "/content/data_multinews"
SPLIT = "test"
LIMIT = 1000
SAVE_PATH = f"/content/pegasus_multidoc_{SPLIT}.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------------- MODEL ----------------
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
model.eval()

# ---------------- LOAD DATA ----------------
src_file, tgt_file = None, None
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.startswith(SPLIT) and "src" in f:
            src_file = os.path.join(root, f)
        if f.startswith(SPLIT) and "tgt" in f:
            tgt_file = os.path.join(root, f)

articles = open(src_file).read().splitlines()[:LIMIT]
references = open(tgt_file).read().splitlines()[:LIMIT]

# ---------------- RESUME SUPPORT ----------------
start_idx = 0
if os.path.exists(SAVE_PATH):
    prev_df = pd.read_csv(SAVE_PATH)
    start_idx = len(prev_df)
    print(f"🔁 Resuming from index {start_idx}")

# ---------------- HELPERS ----------------
def split_docs(sample):
    return [d.strip() for d in sample.split("|||||") if d.strip()]

def summarize(texts, max_len, min_len):
    inputs = tokenizer(
        texts,
        max_length=1024,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        ids = model.generate(
            **inputs,
            max_length=max_len,
            min_length=min_len,
            num_beams=4
        )

    out = tokenizer.batch_decode(ids, skip_special_tokens=True)
    del inputs, ids
    torch.cuda.empty_cache(); gc.collect()
    return out

# ---------------- PIPELINE ----------------
results = []

for i in range(start_idx, len(articles)):
    sample = articles[i]
    docs = split_docs(sample)

    # Stage 1: document-level summaries
    doc_summaries = summarize(docs, max_len=256, min_len=60)

    # Stage 2: fusion summary
    fused_text = " ".join(doc_summaries)
    final_summary = summarize([fused_text], max_len=150, min_len=60)[0]

    results.append({
        "document": sample,
        "reference": references[i],
        "summary": final_summary
    })

    print(f"✓ {i+1}/{len(articles)} done")

    # save every sample (safe for Colab crashes)
    pd.DataFrame(results).to_csv(
        SAVE_PATH,
        mode="a",
        header=not os.path.exists(SAVE_PATH),
        index=False
    )
    results.clear()

    torch.cuda.empty_cache(); gc.collect()

print("✅ Summarization complete. Saved to:", SAVE_PATH)


Device: cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔁 Resuming from index 800
✓ 801/1000 done
✓ 802/1000 done
✓ 803/1000 done
✓ 804/1000 done
✓ 805/1000 done
✓ 806/1000 done
✓ 807/1000 done
✓ 808/1000 done
✓ 809/1000 done
✓ 810/1000 done
✓ 811/1000 done
✓ 812/1000 done
✓ 813/1000 done
✓ 814/1000 done
✓ 815/1000 done
✓ 816/1000 done
✓ 817/1000 done
✓ 818/1000 done
✓ 819/1000 done
✓ 820/1000 done
✓ 821/1000 done
✓ 822/1000 done
✓ 823/1000 done
✓ 824/1000 done
✓ 825/1000 done
✓ 826/1000 done
✓ 827/1000 done
✓ 828/1000 done
✓ 829/1000 done
✓ 830/1000 done
✓ 831/1000 done
✓ 832/1000 done
✓ 833/1000 done
✓ 834/1000 done
✓ 835/1000 done
✓ 836/1000 done
✓ 837/1000 done
✓ 838/1000 done
✓ 839/1000 done
✓ 840/1000 done
✓ 841/1000 done
✓ 842/1000 done
✓ 843/1000 done
✓ 844/1000 done
✓ 845/1000 done
✓ 846/1000 done
✓ 847/1000 done
✓ 848/1000 done
✓ 849/1000 done
✓ 850/1000 done
✓ 851/1000 done
✓ 852/1000 done
✓ 853/1000 done
✓ 854/1000 done
✓ 855/1000 done
✓ 856/1000 done
✓ 857/1000 done
✓ 858/1000 done
✓ 859/1000 done
✓ 860/1000 done
✓ 861/1000 don